In [3]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import keras
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.exposure import equalize_hist
from skimage import exposure, filters

In [4]:
def loadData(path):
    listOfTestFiles = os.listdir(path=path)
    train = []
    train_labels = []
    test = []
    test_labels = []
        
        
    print("Training files = ",listOfTestFiles[1:6])
    #For collecting Training data:
    for file in listOfTestFiles[1:6]:
        with open(path+file,'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            train.append(dict[b'data'])
            train_labels.append(dict[b'labels'])

    print(listOfTestFiles[7])
    #for collecting Testing data
    with open(path+listOfTestFiles[7],'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            test.append(dict[b'data'])
            test_labels.append(dict[b'labels'])

    dictData = {}
    dictData['train_data'] = np.reshape(np.array(train),newshape=(np.array(train).shape[0]*np.array(train).shape[1],np.array(train).shape[2]))
    dictData['train_labels'] = np.reshape(np.array(train_labels),newshape=(np.array(train_labels).shape[0]*np.array(train_labels).shape[1]))
    dictData['test_data'] = np.reshape(np.array(test),newshape=(np.array(test).shape[0]*np.array(test).shape[1],np.array(test).shape[2]))
    dictData['test_labels'] = np.reshape(np.array(test_labels),newshape=(np.array(test_labels).shape[0]*np.array(test_labels).shape[1]))
    return dictData

In [5]:
class kNearestNeighbour(object):
    def __init__(self, metric='l1'):
        """
        Initialize the KNN classifier.
        :param metric: The distance metric to use ('l1', 'l2', 'cosine').
        """
        self.metric = metric

    def train(self, X, Y):
        """
        Memorize the training data.
        :param X: Training data of shape (N, F).
        :param Y: Training labels of shape (N,).
        """
        self.Xtr = X
        self.Ytr = Y

    def _compute_distance(self, x1, x2):
        """
        Compute the distance between a single test example and all training examples.
        :param x1: A single test example of shape (F,).
        :param x2: Training examples of shape (N, F).
        :return: Distance of shape (N,).
        """
        if self.metric == 'l1':
            return np.sum(np.abs(x2 - x1), axis=1)
        elif self.metric == 'l2':
            return np.sqrt(np.sum((x2 - x1) ** 2, axis=1))
        elif self.metric == 'cosine':
            x1_norm = np.linalg.norm(x1)
            x2_norms = np.linalg.norm(x2, axis=1)
            return 1 - (np.dot(x2, x1) / (x1_norm * x2_norms))
        else:
            return np.sum(np.abs(x2 - x1), axis=1) # default L1 distance

    def predict(self, X, k):
        """
        Predict labels for test data.
        :param X: Test data of shape (M, F).
        :param k: Number of neighbors to consider.
        :return: Predicted labels of shape (M,).
        """
        if k > self.Xtr.shape[0]:
            raise ValueError(f"k={k} is greater than the number of training samples={self.Xtr.shape[0]}")

        test_samples = X.shape[0]
        Ypred = np.zeros(test_samples, dtype=self.Ytr.dtype)

        for i in range(test_samples):
            print(f"Test example = {i}", end="\r")

            # Compute distances based on the chosen metric
            dist = self._compute_distance(X[i, :], self.Xtr)

            # Find the indices of the k smallest distances
            idx = np.argpartition(dist, k)[:k]

            # Weighted voting or simple majority voting
            label_count = np.zeros(10, dtype=np.float64)
            for x in idx:
                weight = 1 / (dist[x] + 1e-9)  # Avoid division by zero
                label_count[int(self.Ytr[x])] += weight

            # Assign the label with the maximum weighted vote
            Ypred[i] = np.argmax(label_count)

        return Ypred


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [7]:
# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the data
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # (50000, 32*32*3)
x_test_flat = x_test.reshape(x_test.shape[0], -1)    # (10000, 32*32*3)

# Reshape labels to 1D
y_train = y_train.flatten()
y_test = y_test.flatten()

In [8]:
# Initialize the KNN classifier
knn = kNearestNeighbour()

# Train the classifier
knn.train(x_train_flat, y_train)

# Predict the labels for a subset of test data (e.g., 100 samples)
num_test_samples = 100  # To save time, use a small subset for testing
y_pred = knn.predict(x_test_flat[:num_test_samples], k=20)

# Calculate accuracy
accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.37 99


In [9]:
# Apply LDA
lda = LinearDiscriminantAnalysis(n_components=9)  # CIFAR-10 has 10 classes, so max components = 10 - 1
x_train_lda = lda.fit_transform(x_train_flat, y_train)
x_test_lda = lda.transform(x_test_flat)

In [10]:
# Train the classifier after LDA

knn.train(x_train_lda, y_train)

# Define the range of k values to test
k_values = range(1, 75) 

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_lda[:num_test_samples], k=k)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")


Accuracy for k = 1: 30.00%
Accuracy for k = 2: 30.00%
Accuracy for k = 3: 34.00%
Accuracy for k = 4: 38.00%
Accuracy for k = 5: 38.00%
Accuracy for k = 6: 35.00%
Accuracy for k = 7: 40.00%
Accuracy for k = 8: 39.00%
Accuracy for k = 9: 40.00%
Accuracy for k = 10: 37.00%
Accuracy for k = 11: 38.00%
Accuracy for k = 12: 36.00%
Accuracy for k = 13: 36.00%
Accuracy for k = 14: 33.00%
Accuracy for k = 15: 36.00%
Accuracy for k = 16: 35.00%
Accuracy for k = 17: 34.00%
Accuracy for k = 18: 33.00%
Accuracy for k = 19: 37.00%
Accuracy for k = 20: 37.00%
Accuracy for k = 21: 37.00%
Accuracy for k = 22: 36.00%
Accuracy for k = 23: 37.00%
Accuracy for k = 24: 37.00%
Accuracy for k = 25: 36.00%
Accuracy for k = 26: 38.00%
Accuracy for k = 27: 37.00%
Accuracy for k = 28: 38.00%
Accuracy for k = 29: 38.00%
Accuracy for k = 30: 39.00%
Accuracy for k = 31: 41.00%
Accuracy for k = 32: 39.00%
Accuracy for k = 33: 40.00%
Accuracy for k = 34: 41.00%
Accuracy for k = 35: 41.00%
Accuracy for k = 36: 41.00%
A

In [11]:
# Train the classifier after LDA
knn = kNearestNeighbour(metric='cosine')
knn.train(x_train_lda, y_train)

# Define the range of k values to test
k_values = range(1, 75) 

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_lda[:num_test_samples], k=k)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")

Accuracy for k = 1: 22.00%
Accuracy for k = 2: 22.00%
Accuracy for k = 3: 27.00%
Accuracy for k = 4: 33.00%
Accuracy for k = 5: 33.00%
Accuracy for k = 6: 36.00%
Accuracy for k = 7: 35.00%
Accuracy for k = 8: 35.00%
Accuracy for k = 9: 37.00%
Accuracy for k = 10: 37.00%
Accuracy for k = 11: 35.00%
Accuracy for k = 12: 35.00%
Accuracy for k = 13: 36.00%
Accuracy for k = 14: 38.00%
Accuracy for k = 15: 40.00%
Accuracy for k = 16: 37.00%
Accuracy for k = 17: 40.00%
Accuracy for k = 18: 41.00%
Accuracy for k = 19: 41.00%
Accuracy for k = 20: 38.00%
Accuracy for k = 21: 40.00%
Accuracy for k = 22: 39.00%
Accuracy for k = 23: 40.00%
Accuracy for k = 24: 41.00%
Accuracy for k = 25: 41.00%
Accuracy for k = 26: 41.00%
Accuracy for k = 27: 40.00%
Accuracy for k = 28: 41.00%
Accuracy for k = 29: 41.00%
Accuracy for k = 30: 41.00%
Accuracy for k = 31: 41.00%
Accuracy for k = 32: 41.00%
Accuracy for k = 33: 41.00%
Accuracy for k = 34: 41.00%
Accuracy for k = 35: 41.00%
Accuracy for k = 36: 41.00%
A

In [12]:
# Apply PCA
n_components = 100  # Set the number of components you want to keep after PCA
pca = PCA(n_components=n_components)

# Fit PCA on training data and transform both train and test data
x_train_pca = pca.fit_transform(x_train_flat)
x_test_pca = pca.transform(x_test_flat)

In [13]:
# Flatten y_test if necessary
y_test = y_test.flatten()

# Train the classifier after PCA
knn = kNearestNeighbour(metric='cosine')
knn.train(x_train_pca, y_train)

# Define the range of k values to test
k_values = range(1, 75)

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_pca[:num_test_samples], k=k)  # Use x_test_pca here
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")


Accuracy for k = 1: 38.00%
Accuracy for k = 2: 38.00%
Accuracy for k = 3: 38.00%
Accuracy for k = 4: 43.00%
Accuracy for k = 5: 42.00%
Accuracy for k = 6: 41.00%
Accuracy for k = 7: 44.00%
Accuracy for k = 8: 44.00%
Accuracy for k = 9: 44.00%
Accuracy for k = 10: 38.00%
Accuracy for k = 11: 46.00%
Accuracy for k = 12: 46.00%
Accuracy for k = 13: 45.00%
Accuracy for k = 14: 47.00%
Accuracy for k = 15: 49.00%
Accuracy for k = 16: 50.00%
Accuracy for k = 17: 48.00%
Accuracy for k = 18: 46.00%
Accuracy for k = 19: 48.00%
Accuracy for k = 20: 48.00%
Accuracy for k = 21: 47.00%
Accuracy for k = 22: 49.00%
Accuracy for k = 23: 50.00%
Accuracy for k = 24: 52.00%
Accuracy for k = 25: 50.00%
Accuracy for k = 26: 50.00%
Accuracy for k = 27: 49.00%
Accuracy for k = 28: 51.00%
Accuracy for k = 29: 50.00%
Accuracy for k = 30: 50.00%
Accuracy for k = 31: 51.00%
Accuracy for k = 32: 50.00%
Accuracy for k = 33: 51.00%
Accuracy for k = 34: 50.00%
Accuracy for k = 35: 48.00%
Accuracy for k = 36: 50.00%
A

In [14]:
def gabor_features(images, kernels=None):
    """
    Apply Gabor filters to a batch of images.
    
    Parameters:
    - images: A batch of images of shape (N, H, W, C) where N is the number of images,
      H is height, W is width, and C is the number of channels.
    - kernels: A list of Gabor kernels to apply. If None, it generates a set of kernels.
    
    Returns:
    - features: A 2D array of shape (N, F) where F is the number of features per image.
    """
    if kernels is None:
        kernels = generate_gabor_kernels()
    
    # Initialize a list to store the features
    features = []
    
    for img in images:
        # Ensure the image is grayscale (Gabor requires single-channel images)
        if img.ndim == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        img_features = []
        for kernel in kernels:
            # Apply the Gabor filter
            filtered = cv2.filter2D(img, cv2.CV_32F, kernel)
            # Extract statistics as features (mean and standard deviation)
            img_features.append(filtered.mean())
            img_features.append(filtered.std())
        
        features.append(img_features)
    
    return np.array(features)

def generate_gabor_kernels(scales=5, orientations=8):
    """
    Generate Gabor filter kernels.
    
    Parameters:
    - scales: Number of different scales (frequencies).
    - orientations: Number of orientations for each scale.
    
    Returns:
    - kernels: A list of Gabor kernels.
    """
    kernels = []
    for theta in np.linspace(0, np.pi, orientations, endpoint=False):  # Orientations
        for sigma in (1, 2):  # Scale parameters
            kernel = cv2.getGaborKernel(ksize=(9, 9), sigma=sigma, theta=theta, lambd=10.0, gamma=0.5, psi=0)
            kernels.append(kernel)
    return kernels

def generate_features_in_batches(data, batch_size, feature_extractor):
    """
    Generate features in batches to save memory.
    :param data: Original data of shape (N, H, W, C).
    :param batch_size: Batch size for feature computation.
    :param feature_extractor: A callable that extracts features for a batch.
    :return: Features array.
    """
    features = []
    for i in range(0, data.shape[0], batch_size):
        print(f"Processing batch {i // batch_size + 1}", end="\r")
        batch = data[i:i + batch_size]
        batch_features = feature_extractor(batch)  # Apply feature extraction
        features.append(batch_features)
    return np.vstack(features)


In [15]:
kernels = generate_gabor_kernels()

x_train_gabor = generate_features_in_batches(
    x_train, batch_size=1000, feature_extractor=lambda x: gabor_features(x, kernels)
)
x_test_gabor = generate_features_in_batches(
    x_test, batch_size=1000, feature_extractor=lambda x: gabor_features(x, kernels)
)

In [16]:
def normalize_features(features):
    return (features - features.mean(axis=0)) / features.std(axis=0)

x_train_gabor = normalize_features(x_train_gabor)
x_test_gabor = normalize_features(x_test_gabor)

In [17]:
# Flatten y_test if necessary
y_test = y_test.flatten()

knn = kNearestNeighbour(metric='cosine')
knn.train(x_train_gabor, y_train)

# Define the range of k values to test
k_values = range(1, 75)

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_gabor[:num_test_samples], k=k)  # Use x_test_pca here
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")

y_pred_full = knn.predict(x_test_gabor, k=best_k)
final_accuracy = accuracy_score(y_test, y_pred_full)
print(f"Final accuracy on the full test set: {final_accuracy * 100:.2f}%")


Accuracy for k = 1: 18.00%
Accuracy for k = 2: 18.00%
Accuracy for k = 3: 18.00%
Accuracy for k = 4: 20.00%
Accuracy for k = 5: 18.00%
Accuracy for k = 6: 17.00%
Accuracy for k = 7: 19.00%
Accuracy for k = 8: 18.00%
Accuracy for k = 9: 18.00%
Accuracy for k = 10: 22.00%
Accuracy for k = 11: 21.00%
Accuracy for k = 12: 20.00%
Accuracy for k = 13: 21.00%
Accuracy for k = 14: 21.00%
Accuracy for k = 15: 23.00%
Accuracy for k = 16: 25.00%
Accuracy for k = 17: 24.00%
Accuracy for k = 18: 24.00%
Accuracy for k = 19: 23.00%
Accuracy for k = 20: 24.00%
Accuracy for k = 21: 24.00%
Accuracy for k = 22: 24.00%
Accuracy for k = 23: 23.00%
Accuracy for k = 24: 23.00%
Accuracy for k = 25: 25.00%
Accuracy for k = 26: 26.00%
Accuracy for k = 27: 26.00%
Accuracy for k = 28: 25.00%
Accuracy for k = 29: 27.00%
Accuracy for k = 30: 26.00%
Accuracy for k = 31: 26.00%
Accuracy for k = 32: 28.00%
Accuracy for k = 33: 27.00%
Accuracy for k = 34: 27.00%
Accuracy for k = 35: 25.00%
Accuracy for k = 36: 27.00%
A

In [18]:
def compute_phog(image):
    """
    Compute the Pyramid Histogram of Oriented Gradients (PHOG) features for an image.

    Parameters:
    - image: Input image (2D grayscale or 3D color).

    Returns:
    - fd: Flattened feature descriptor for the image.
    """
    # Convert to grayscale if the image is in color
    if image.ndim == 3:
        image = rgb2gray(image)
    
    # Compute HOG features
    fd = hog(
        image, 
        orientations=9, 
        pixels_per_cell=(8, 8), 
        cells_per_block=(1, 1), 
        channel_axis=None  # Specify no channel for grayscale images
    )
    return fd

In [19]:
def preprocess_histogram_equalization(image):
    """
    Apply histogram equalization to enhance image contrast.

    Parameters:
    - image: Input image (2D grayscale or 3D color).

    Returns:
    - processed_image: Image after histogram equalization.
    """
    if image.ndim == 3:  # Convert to grayscale if necessary
        image = rgb2gray(image)
    return equalize_hist(image)

# Apply to dataset
x_train_preprocessed = np.array([preprocess_histogram_equalization(img) for img in x_train])
x_test_preprocessed = np.array([preprocess_histogram_equalization(img) for img in x_test])

In [20]:
def preprocess_clahe(image):
    """
    Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better local contrast.

    Parameters:
    - image: Input image (2D grayscale or 3D color).

    Returns:
    - processed_image: Image after CLAHE.
    """
    if image.ndim == 3:  # Convert to grayscale if necessary
        image = rgb2gray(image)
        image = (image * 255).astype(np.uint8)  # Convert to uint8 for CLAHE
    
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Apply to dataset
x_train_preprocessed = np.array([preprocess_clahe(img) for img in x_train])
x_test_preprocessed = np.array([preprocess_clahe(img) for img in x_test])

In [21]:
def preprocess_intensity_normalization(image):
    """
    Normalize pixel intensities to the range [0, 1].

    Parameters:
    - image: Input image (2D grayscale or 3D color).

    Returns:
    - processed_image: Normalized image.
    """
    if image.ndim == 3:  # Convert to grayscale if necessary
        image = rgb2gray(image)
    return (image - image.min()) / (image.max() - image.min())

# Apply to dataset
x_train_preprocessed = np.array([preprocess_intensity_normalization(img) for img in x_train])
x_test_preprocessed = np.array([preprocess_intensity_normalization(img) for img in x_test])


In [22]:
x_train_phog = np.array([compute_phog(img) for img in x_train_preprocessed])
x_test_phog = np.array([compute_phog(img) for img in x_test_preprocessed])

In [23]:
# Flatten y_test if necessary
y_test = y_test.flatten()

# Train the classifier after PCA
knn = kNearestNeighbour(metric='cosine')
knn.train(x_train_phog, y_train)

# Define the range of k values to test
k_values = range(1, 75)

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_phog[:num_test_samples], k=k)  # Use x_test_pca here
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")

Accuracy for k = 1: 28.00%
Accuracy for k = 2: 28.00%
Accuracy for k = 3: 33.00%
Accuracy for k = 4: 35.00%
Accuracy for k = 5: 35.00%
Accuracy for k = 6: 34.00%
Accuracy for k = 7: 31.00%
Accuracy for k = 8: 35.00%
Accuracy for k = 9: 34.00%
Accuracy for k = 10: 33.00%
Accuracy for k = 11: 34.00%
Accuracy for k = 12: 31.00%
Accuracy for k = 13: 32.00%
Accuracy for k = 14: 34.00%
Accuracy for k = 15: 34.00%
Accuracy for k = 16: 36.00%
Accuracy for k = 17: 33.00%
Accuracy for k = 18: 31.00%
Accuracy for k = 19: 34.00%
Accuracy for k = 20: 36.00%
Accuracy for k = 21: 33.00%
Accuracy for k = 22: 33.00%
Accuracy for k = 23: 34.00%
Accuracy for k = 24: 34.00%
Accuracy for k = 25: 34.00%
Accuracy for k = 26: 34.00%
Accuracy for k = 27: 33.00%
Accuracy for k = 28: 32.00%
Accuracy for k = 29: 32.00%
Accuracy for k = 30: 34.00%
Accuracy for k = 31: 34.00%
Accuracy for k = 32: 34.00%
Accuracy for k = 33: 32.00%
Accuracy for k = 34: 31.00%
Accuracy for k = 35: 31.00%
Accuracy for k = 36: 31.00%
A

In [24]:
## WHAT IF PRE PROCESSING + GABOR + KNN 

x_train_preprocessed = np.array([preprocess_histogram_equalization(img) for img in x_train_flat])
x_test_preprocessed = np.array([preprocess_histogram_equalization(img) for img in x_test_flat])

x_train_gabor_pre = generate_features_in_batches(
    x_train_preprocessed, batch_size=1000, feature_extractor=lambda x: gabor_features(x, kernels)
)
x_test_gabor_pre = generate_features_in_batches(
    x_test_preprocessed, batch_size=1000, feature_extractor=lambda x: gabor_features(x, kernels)
)

print(f"x_train_gabor_pre shape: {x_train_gabor_pre.shape}")
print(f"x_test_gabor_pre shape: {x_test_gabor_pre.shape}")

x_train_gabor_pre shape: (50000, 32)
x_test_gabor_pre shape: (10000, 32)


In [25]:
n_components = 32  
pca = PCA(n_components=n_components)

x_train_preprocessed_1 = np.array([preprocess_intensity_normalization(img) for img in x_train_flat])
x_test_preprocessed_1 = np.array([preprocess_intensity_normalization(img) for img in x_test_flat])

x_train_final = pca.fit_transform(x_train_preprocessed_1)
x_test_final = pca.transform(x_test_preprocessed_1)

In [26]:
# Flatten y_test if necessary
y_test = y_test.flatten()

knn = kNearestNeighbour(metric='cosine')
knn.train(x_train_final, y_train)

# Define the range of k values to test
k_values = range(1, 75)

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_final[:num_test_samples], k=k)  # Use x_test_pca here
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")

Accuracy for k = 1: 42.00%
Accuracy for k = 2: 42.00%
Accuracy for k = 3: 46.00%
Accuracy for k = 4: 44.00%
Accuracy for k = 5: 51.00%
Accuracy for k = 6: 47.00%
Accuracy for k = 7: 47.00%
Accuracy for k = 8: 50.00%
Accuracy for k = 9: 50.00%
Accuracy for k = 10: 44.00%
Accuracy for k = 11: 50.00%
Accuracy for k = 12: 47.00%
Accuracy for k = 13: 46.00%
Accuracy for k = 14: 48.00%
Accuracy for k = 15: 50.00%
Accuracy for k = 16: 50.00%
Accuracy for k = 17: 49.00%
Accuracy for k = 18: 50.00%
Accuracy for k = 19: 49.00%
Accuracy for k = 20: 49.00%
Accuracy for k = 21: 49.00%
Accuracy for k = 22: 50.00%
Accuracy for k = 23: 50.00%
Accuracy for k = 24: 51.00%
Accuracy for k = 25: 52.00%
Accuracy for k = 26: 49.00%
Accuracy for k = 27: 48.00%
Accuracy for k = 28: 47.00%
Accuracy for k = 29: 48.00%
Accuracy for k = 30: 46.00%
Accuracy for k = 31: 46.00%
Accuracy for k = 32: 47.00%
Accuracy for k = 33: 47.00%
Accuracy for k = 34: 46.00%
Accuracy for k = 35: 47.00%
Accuracy for k = 36: 50.00%
A

In [27]:
class kNearestNeighbourKernel(object):
    def __init__(self, metric='l1', kernel='uniform', bandwidth=1.0):
        """
        Initialize the KNN classifier.
        :param metric: The distance metric to use ('l1', 'l2', 'cosine').
        :param kernel: The kernel function to use ('uniform', 'gaussian', 'epanechnikov').
        :param bandwidth: The bandwidth parameter for kernel functions.
        """
        self.metric = metric
        self.kernel = kernel
        self.bandwidth = bandwidth

    def train(self, X, Y):
        """
        Memorize the training data.
        :param X: Training data of shape (N, F).
        :param Y: Training labels of shape (N,).
        """
        self.Xtr = X
        self.Ytr = Y

    def _compute_distance(self, x1, x2):
        """
        Compute the distance between a single test example and all training examples.
        :param x1: A single test example of shape (F,).
        :param x2: Training examples of shape (N, F).
        :return: Distance of shape (N,).
        """
        if self.metric == 'l1':
            return np.sum(np.abs(x2 - x1), axis=1)
        elif self.metric == 'l2':
            return np.sqrt(np.sum((x2 - x1) ** 2, axis=1))
        elif self.metric == 'cosine':
            x1_norm = np.linalg.norm(x1)
            x2_norms = np.linalg.norm(x2, axis=1)
            return 1 - (np.dot(x2, x1) / (x1_norm * x2_norms))
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")

    def _compute_kernel(self, distances):
        """
        Apply the kernel function to the distances.
        :param distances: Distances of shape (N,).
        :return: Kernel weights of shape (N,).
        """
        if self.kernel == 'uniform':
            return np.ones_like(distances)
        elif self.kernel == 'gaussian':
            return np.exp(-0.5 * (distances / self.bandwidth) ** 2)
        elif self.kernel == 'epanechnikov':
            weights = np.maximum(0, 1 - (distances / self.bandwidth) ** 2)
            return weights
        else:
            raise ValueError(f"Unsupported kernel: {self.kernel}")

    def predict(self, X, k):
        """
        Predict labels for test data.
        :param X: Test data of shape (M, F).
        :param k: Number of neighbors to consider.
        :return: Predicted labels of shape (M,).
        """
        if k > self.Xtr.shape[0]:
            raise ValueError(f"k={k} is greater than the number of training samples={self.Xtr.shape[0]}")

        test_samples = X.shape[0]
        Ypred = np.zeros(test_samples, dtype=self.Ytr.dtype)

        for i in range(test_samples):
            print(f"Processing test sample {i + 1}/{test_samples}", end="\r")

            # Compute distances based on the chosen metric
            dist = self._compute_distance(X[i, :], self.Xtr)

            # Find the indices of the k smallest distances
            idx = np.argpartition(dist, k)[:k]

            # Compute kernel weights for the selected neighbors
            neighbor_distances = dist[idx]
            weights = self._compute_kernel(neighbor_distances)

            # Weighted voting
            label_count = np.zeros(10, dtype=np.float64)  # Assuming 10 classes
            for j, x in enumerate(idx):
                label_count[int(self.Ytr[x])] += weights[j]

            # Assign the label with the maximum weighted vote
            Ypred[i] = np.argmax(label_count)

        return Ypred


In [35]:
# Flatten y_test if necessary
y_test = y_test.flatten()

knn = kNearestNeighbourKernel(metric='cosine', kernel='epanechnikov', bandwidth=5)
knn.train(x_train_final, y_train)

# Define the range of k values to test
k_values = range(1, 75)

# Store the accuracies for each k
accuracies = []

best_k = None
highest_accuracy = 0

for k in k_values:
    # Predict the labels for a subset of test data
    num_test_samples = 100  # To save time, use a small subset for testing
    y_pred = knn.predict(x_test_final[:num_test_samples], k=k)  # Use x_test_pca here
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test[:num_test_samples], y_pred)
    accuracies.append(accuracy)
    
    # Check if this is the best accuracy so far
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        best_k = k
    
    # Print accuracy for this k
    print(f"Accuracy for k = {k}: {accuracy * 100:.2f}%")

# Print the best k and highest accuracy
print("\nBest k value:")
print(f"k = {best_k}: Accuracy = {highest_accuracy * 100:.2f}%")

Accuracy for k = 1: 42.00%/100
Accuracy for k = 2: 42.00%/100
Accuracy for k = 3: 46.00%/100
Accuracy for k = 4: 44.00%/100
Accuracy for k = 5: 50.00%/100
Accuracy for k = 6: 47.00%/100
Accuracy for k = 7: 47.00%/100
Accuracy for k = 8: 49.00%/100
Accuracy for k = 9: 47.00%/100
Accuracy for k = 10: 43.00%100
Accuracy for k = 11: 48.00%100
Accuracy for k = 12: 46.00%100
Accuracy for k = 13: 45.00%100
Accuracy for k = 14: 47.00%100
Accuracy for k = 15: 49.00%100
Accuracy for k = 16: 49.00%100
Accuracy for k = 17: 49.00%100
Accuracy for k = 18: 50.00%100
Accuracy for k = 19: 47.00%100
Accuracy for k = 20: 48.00%100
Accuracy for k = 21: 47.00%100
Accuracy for k = 22: 49.00%100
Accuracy for k = 23: 48.00%100
Accuracy for k = 24: 49.00%100
Accuracy for k = 25: 51.00%100
Accuracy for k = 26: 48.00%100
Accuracy for k = 27: 46.00%100
Accuracy for k = 28: 45.00%100
Accuracy for k = 29: 46.00%100
Accuracy for k = 30: 46.00%100
Accuracy for k = 31: 46.00%100
Accuracy for k = 32: 47.00%100
Accuracy